## Short Summary
Script name: product_matcher.py

Purpose:

Checks if a product description (from a PDF or other source) has already been processed and stored in the database—so you don’t repeat expensive extraction/AI work.

Uses both exact matching (via hashes) and fuzzy matching (similarity threshold) for robust duplicate detection.

Saves new results to cache after processing, so future requests are faster.

Meant for use as a pre-processing step in a pipeline before you run costly scrapers, LLMs, or APIs.

# Product Cache Matching & Fuzzy Duplicate Detection

This notebook contains tools to check whether a product description from your PDF or ETL workflow has already been processed.  
It avoids duplicate processing using both exact (hash-based) and fuzzy (string-similarity) matching against your SQLite cache.


## Prerequisites

- Python 3.8+
- `sqlite3` (built-in)
- `difflib` (built-in, for string similarity)
- `database_setup.py` in your working directory (to import the hash generator)


Core Class with Dogstring and Comments (Code)

In [ ]:
import sqlite3
from datetime import datetime
from database_setup import create_description_hash
from difflib import SequenceMatcher

class ProductMatcher:
    """
    Class to manage product caching and duplicate detection using SQLite.
    """

    def __init__(self, db_path='product_cache.db'):
        self.db_path = db_path

    def get_connection(self):
        "Helper to get SQLite DB connection."
        return sqlite3.connect(self.db_path)

    def check_cache(self, oz_number, description, menge=""):
        """
        Checks for an **exact match** in the cache using a hash.
        Returns product info if found, otherwise returns not found.
        """
        desc_hash = create_description_hash(oz_number, description, menge)
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('''
            SELECT * FROM extraction_cache 
            WHERE description_hash = ?
        ''', (desc_hash,))
        result = cursor.fetchone()
        if result:
            # Update last_accessed timestamp
            cursor.execute('''
                UPDATE extraction_cache 
                SET last_accessed = CURRENT_TIMESTAMP 
                WHERE description_hash = ?
            ''', (desc_hash,))
            conn.commit()
            print(f"✅ Found in cache: {result[5]} - {result[6]}")
            conn.close()
            return {
                'found': True,
                'scraped_brand': result[5],
                'scraped_product_name': result[6], 
                'scraped_product_description': result[7],
                'scraped_price': result[8],
                'standard_category': result[9] if len(result) > 9 else 'Other',
                'method': 'cache'
            }
        conn.close()
        return {'found': False}

    def fuzzy_search_similar(self, description):
        """
        Finds **similar** product descriptions using fuzzy string matching.
        Only considers previous extractions with valid brands.
        Returns highest-confidence match above 70% similarity.
        """
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('''
            SELECT original_description, scraped_brand, scraped_product_name, 
                   scraped_product_description, scraped_price, standard_category
            FROM extraction_cache
            WHERE scraped_brand NOT IN ('No Results', 'Processing Error', 'All Scraping Failed')
        ''')
        cached_items = cursor.fetchall()
        matches = []
        for item in cached_items:
            cached_desc, brand, name, desc, price, category = item
            similarity = SequenceMatcher(None, description.lower(), cached_desc.lower()).ratio()
            if similarity > 0.7:
                matches.append({
                    'similarity': similarity,
                    'scraped_brand': brand,
                    'scraped_product_name': name,
                    'scraped_product_description': desc,
                    'scraped_price': price,
                    'standard_category': category or 'Other',
                    'original_description': cached_desc
                })
        conn.close()
        if matches:
            best_match = max(matches, key=lambda x: x['similarity'])
            print(f"📊 Found similar item: {best_match['similarity']:.1%} match")
            print(f"   Original: {best_match['original_description'][:50]}...")
            print(f"   Product: {best_match['scraped_brand']} - {best_match['scraped_product_name']}")
            return {
                'found': True,
                'method': 'fuzzy_match',
                'confidence': best_match['similarity'],
                **{k: v for k, v in best_match.items() if k != 'similarity'}
            }
        return {'found': False}

    def save_to_cache(self, oz_number, description, menge, scraped_data):
        """
        Saves product extraction results to cache.
        Can overwrite existing records for that hash.
        """
        desc_hash = create_description_hash(oz_number, description, menge)
        conn = self.get_connection()
        cursor = conn.cursor()
        cursor.execute('''
            INSERT OR REPLACE INTO extraction_cache 
            (oz_number, description_hash, original_description, menge,
             scraped_brand, scraped_product_name, scraped_product_description, scraped_price, standard_category)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            oz_number, desc_hash, description, menge,
            scraped_data.get('scraped_brand', ''),
            scraped_data.get('scraped_product_name', ''),
            scraped_data.get('scraped_product_description', ''),
            scraped_data.get('scraped_price', ''),
            scraped_data.get('standard_category', 'Other')
        ))
        conn.commit()
        conn.close()
        print("💾 Saved to cache for future use")

    def check_before_processing(self, oz_number, description, menge=""):
        """
        MAIN FUNCTION: Use before running expensive AI or scraping.
        1. Checks for exact cache match.
        2. If not found, checks for high-similarity fuzzy match.
        3. If nothing is found, signals that new processing is needed.
        """
        print(f"\n🔎 Checking: {oz_number}")
        print(f"   Description: {description[:60]}...")
        print(f"   Menge: {menge}")

        # Step 1: Exact cache check
        cache_result = self.check_cache(oz_number, description, menge)
        if cache_result['found']:
            return cache_result

        # Step 2: Similar description check  
        fuzzy_result = self.fuzzy_search_similar(description)
        if fuzzy_result['found'] and fuzzy_result['confidence'] > 0.8:
            print("🎯 High confidence match - using cached result")
            # Save this as exact match for future
            self.save_to_cache(oz_number, description, menge, fuzzy_result)
            return fuzzy_result

        # Step 3: Not found - need to process
        print("❌ Not found in cache - need to process with your script")
        return {'found': False, 'message': 'Process with your existing logic'}


## Example: Check and Cache a Product Extraction


In [ ]:
matcher = ProductMatcher()

# Simulate a new product extraction line from your PDF/workflow
result = matcher.check_before_processing(
    oz_number="01.02.2024",
    description="Wireless Bluetooth headphones with noise cancellation", 
    menge="5 Stk"
)
print(f"Result: {result}")


💡 **Tip:**  
Always run this check before invoking your main (expensive) AI extraction!  
If a match is found, you avoid unnecessary API calls.


# Summary for Externals

1. What does this do?

Provides duplicate detection for your product extraction workflow.
If an entry is already in your cache (exact or fuzzy match), you can reuse its data and save costs.

2. Who should use it?

Anyone running batch ETL, AI product extraction, or web scraping jobs that could create duplicates.

3. Why is it important?

Saves money, time, and avoids redundant work. Ensures clean, unique product data.